# IERG4190 Multimedia Coding and Processing
---
## Chapter 3 Multimedia Coding (2)
[Lecture Note](https://blackboard.cuhk.edu.hk/bbcswebdav/pid-3095878-dt-content-rid-23712435_1/xid-23712435_1) (Need CUHK Logon)

Multi-symbol Coding with Adaptive Dictionary
### LZ77
#### Basic Information
In the LZ77 approach, the dictionary is simply a portion of the previously encoded sequence. The encoder examines the input sequence through a sliding window.

*See the graphical explanation on slide 5 in the lecture note.*

Search buffer: contains a portion of the recently encoded sequence, with usually several thousand characters.

Look-ahead buffer: contains the next portion of the sequence to be encoded, with usually ten to one hundred characters.

To encode the sequence in the look-ahead buffer, the encoder moves a search pointer back through the search buffer until it encounters a match to the first symbol in the look-ahead buffer.

Offset: the distance of the pointer from the look-ahead buffer.

Length-of-match: the number of consecutive symbols in the search buffer that match consecutive symbols in the look-ahead buffer, starting with the first symbol.

Once the longest match is found, the encoder encodes a triple:

<the offset, the length of match, the symbol following the match>

*See the graphical explanation on slide 6 in the lecture note.*


#### Example
*See slides 7-11 in the lecture note.*

#### Limitations
Very simple adaptive scheme that requires no any prior knowledge of the source. The dictionary is the search window.

Problems with LZ77:
* The algorithm uses only a small window into previously seen text, which means it continuously throws away valuable phrases because they slide out of the dictionary.
* The limited lengths of the two buffers limit the size of a phrase that can be matched.
* Worst case situation is that the sequence to be encoded is periodic with a period longer than the search buffer.
    > e.g. **abcdefghij**kabcdefghijk
*Note that the following code may have bugs under some situations!*

In [12]:
# coding=utf-8
# sample code from https://blog.csdn.net/yezhen910328/article/details/51852590
 
class LZ77:
    """
    A simplified implementation of LZ77 algorithm
    """
 
    def __init__(self, window_size):
        self.window_size = window_size
        self.buffer_size = 4
 
    def longest_match(self, data, cursor):
        """
        find the longest match between in dictionary and lookahead-buffer
        """
        end_buffer = min(cursor + self.buffer_size, len(data))
 
        p = -1
        l = -1
        c = ''
 
        for j in range(cursor+1, end_buffer+1):
            start_index = max(0, cursor - self.window_size + 1)
            substring = data[cursor + 1:j + 1]
 
            for i in range(start_index, cursor+1):
                repetition = (int)(len(substring) / (cursor - i + 1))
                last = len(substring) % (cursor - i + 1)
                matchedstring = data[i:cursor + 1] * repetition + data[i:i + last]
 
                if matchedstring == substring and len(substring) > l:
                    p = cursor - i + 1
                    l = len(substring)
                    c = data[j+1]
 
        # unmatched string between the two
        if p == -1 and l == -1:
            return 0, 0, data[cursor + 1]
        return p, l, c
 
    def compress(self, message):
        """
        compress message
        :return: tuples (p, l, c)
        """
        i = -1
        out = []
 
        # the cursor move until it reaches the end of message
        while i < len(message)-1:
            (p, l, c) = self.longest_match(message, i)
            out.append((p, l, c))
            i += (l+1)
        return out
 
    def decompress(self, compressed):
        """
        decompress the compressed message
        :param compressed: tuples (p, l, c)
        :return: decompressed message
        """
        cursor = -1
        out = ''
 
        for (p, l, c) in compressed:
            # the initialization
            if p == 0 and l == 0:
                out += c
            elif p >= l:
                out += (out[cursor-p+1:cursor+1] + c)
 
            # the repetition of dictionary
            elif p < l:
                repetition = (int)(l / p)
                last = l % p
                out += (out[cursor-p+1:cursor+1] * repetition + out[cursor-p+1:last] + c)
            cursor += (l + 1)
 
        return out
 
 
if __name__ == '__main__':
    compressor = LZ77(8)
    origin = list('aacaacabcabaaac')
    pack = compressor.compress(origin)
    unpack = compressor.decompress(pack)
    print(pack)
    print(unpack)
    print(unpack == 'aacaacabcabaaac')

[(0, 0, 'a'), (1, 1, 'c'), (3, 4, 'b'), (3, 3, 'a'), (1, 2, 'c')]
aacaacabcabaaac
True


### LZ78
#### Comparison between LZ77 and LZ78
LZ77:

The dictionary of phrases is defined by a fixed window of previously seen text.

It outputs a series of tokens (triples):

<the offset, the length of match, the single symbol following the match>

LZ78:

The dictionary is a potentially unlimited list of previously seen phrases. It is built at both the encoder and decoder in an identical manner.

It outputs a series of tokens (doubles):

<the index selecting a given phrase, the single symbol following the phrase>
#### Basic Information
LZ78 codes the input as a double $<i,c>$:

$i$: an index corresponding to the dictionary entry that is the longest match to the input. It's 0 in the case of no match.

$c$: the code for the character in the input following the matched portion of the input.

This double then becomes the newest entry in the dictionary.
> Each new entry into the dictionary is one new symbol concatenated with an existing dictionary entry.

#### Example
*See slides 15-16 in the lecture note.*

In [13]:
# sample code from https://www.cnblogs.com/en-heng/p/6283282.html
def compress(message):
    tree_dict, m_len, i = {}, len(message), 0
    while i < m_len:
        # case I
        if message[i] not in tree_dict.keys():
            yield (0, message[i])
            tree_dict[message[i]] = len(tree_dict) + 1
            i += 1
        # case III
        elif i == m_len - 1:
            yield (tree_dict.get(message[i]), '')
            i += 1
        else:
            for j in range(i + 1, m_len):
                # case II
                if message[i:j + 1] not in tree_dict.keys():
                    yield (tree_dict.get(message[i:j]), message[j])
                    tree_dict[message[i:j + 1]] = len(tree_dict) + 1
                    i = j + 1
                    break
                # case III
                elif j == m_len - 1:
                    yield (tree_dict.get(message[i:j + 1]), '')
                    i = j + 1


def uncompress(packed):
    unpacked, tree_dict = '', {}
    for index, ch in packed:
        if index == 0:
            unpacked += ch
            tree_dict[len(tree_dict) + 1] = ch
        else:
            term = tree_dict.get(index) + ch
            unpacked += term
            tree_dict[len(tree_dict) + 1] = term
    return unpacked


if __name__ == '__main__':
    messages = ['ABBCBCABABCAABCAAB', 'BABAABRRRA', 'AAAAAAAAA']
    for m in messages:
        pack = compress(m)
        unpack = uncompress(pack)
        print(unpack == m)

True
True
True


### Notes on LZ77 & LZ78
* Both encoder and decoder will come up with same dictionary   
    * No dictionary has to be sent
* LZ77 not necessarily inferior
    * Used with Huffman Coding in DEFLATE
    * DEFLATE is used in many applications, including originally in PKZip (general purpose) and eventually become a compression standard ("zip")
    * Also in a very modern standard, PNG (image compression)